In [15]:
import numpy as np
import pandas as pd
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [8]:
with open(r"C:\Users\Antho\.secret\api-key.json") as f:
    login = json.load(f)
yelp_api = YelpAPI(login["api-key"], timeout_s = 5.0)

In [9]:
LOCATION = "Philadelphia, PA"
TERM = "Cheesesteaks"

In [10]:
JSON_FILE = "Data/results_in_progress_Philly_cheeseteaks.json"
JSON_FILE

'Data/results_in_progress_Philly_cheeseteaks.json'

In [11]:
def create_json_file(JSON_FILE, delete_if_exists = True):
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == True:
        if delete_if_exists == True:
            print(f"[!] {JSON_FILE} already exists. Deleting previous file")
            os.remove(JSON_FILE)
            create_json_file(JSON_FILE, delete_if_exists = False)
        else:
            print(f"[i] {JSON_FILE} alread exists.")
    else:
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        folder = os.path.dirname(JSON_FILE)
        if len(folder) > 0:
            os.makedirs(folder, exist_ok = True)
        with open(JSON_FILE, "w") as f:
            json.dump([], f)

In [12]:
create_json_file(JSON_FILE, delete_if_exists = True)
with open(JSON_FILE, "r") as f:
    previous_results = json.load(f)
n_results = len(previous_results)
print(f'-{n_results} previous results found.')
results = yelp_api.search_query(location = LOCATION, term = TERM, 
                               offset = n_results)
total_results = results["total"]
results_per_page = len(results["businesses"])
n_pages = math.ceil((results["total"]-n_results)/results_per_page)
n_pages

[i] Data/results_in_progress_Philly_cheeseteaks.json not found. Saving empty list to new file.
-0 previous results found.


190

In [13]:
for i in tqdm_notebook(range(1, n_pages + 1)):
    with open(JSON_FILE, "r") as f:
        previous_results = json.load(f)
    n_results = len(previous_results)
    if (n_results + results_per_page) > 1000:
        print("Exceeded 1000 api calls. Stopping loop.")
        break
    results = yelp_api.search_query(location = LOCATION, term = TERM, 
                                   offset = n_results)
    previous_results.extend(results["businesses"])
    with open(JSON_FILE, "w") as f:
        json.dump(previous_results, f)
    time.sleep(.2)

  0%|          | 0/190 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [14]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,RQAF6a0akMiot5lZZnMNNw,dalessandro-s-steaks-and-hoagies-philadelphia,Dalessandro’s Steaks & Hoagies,https://s3-media1.fl.yelpcdn.com/bphoto/MRvZIW...,False,https://www.yelp.com/biz/dalessandro-s-steaks-...,2948,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.0,"{'latitude': 40.029494, 'longitude': -75.2059714}",[delivery],$$,"{'address1': '600 Wendover St', 'address2': ''...",+12154825407,(215) 482-5407,8924.599279
1,jxEMFqwDJXjCxmcm5t5jVQ,cleavers-philadelphia,Cleavers,https://s3-media1.fl.yelpcdn.com/bphoto/9dfyW_...,False,https://www.yelp.com/biz/cleavers-philadelphia...,1438,"[{'alias': 'cheesesteaks', 'title': 'Cheeseste...",4.0,"{'latitude': 39.9513931274414, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '108 S 18th St', 'address2': '', ...",+12155153828,(215) 515-3828,268.685385
2,-mIlmp5l4hKlp1tvHRdvTg,sonnys-famous-steaks-philadelphia,Sonny's Famous Steaks,https://s3-media2.fl.yelpcdn.com/bphoto/c333HA...,False,https://www.yelp.com/biz/sonnys-famous-steaks-...,1810,"[{'alias': 'cheesesteaks', 'title': 'Cheeseste...",4.0,"{'latitude': 39.94982923010674, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '228 Market St', 'address2': '', ...",+12156295760,(215) 629-5760,2293.831534
3,LM54ufrINJWoTN5imV8Etw,johns-roast-pork-philadelphia,John's Roast Pork,https://s3-media4.fl.yelpcdn.com/bphoto/C5APCS...,False,https://www.yelp.com/biz/johns-roast-pork-phil...,1680,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.5,"{'latitude': 39.9210273851256, 'longitude': -7...",[delivery],$$,"{'address1': '14 E Snyder Ave', 'address2': ''...",+12154631951,(215) 463-1951,4271.479109
4,7pAgxBMUjrVPH7xh3fn-gw,oh-brother-philly-philadelphia,Oh Brother Philly,https://s3-media3.fl.yelpcdn.com/bphoto/YbjY1z...,False,https://www.yelp.com/biz/oh-brother-philly-phi...,642,"[{'alias': 'cheesesteaks', 'title': 'Cheeseste...",4.5,"{'latitude': 39.94971, 'longitude': -75.14411}","[delivery, pickup]",$$,"{'address1': '206 Market St', 'address2': '', ...",+12155153255,(215) 515-3255,2349.426824


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,10Bq-D7cUzwvY5pUJPWhmw,ralphs-pizza-and-francescas-room-haddon-heights,Ralph's Pizza & Francesca's Room,https://s3-media4.fl.yelpcdn.com/bphoto/TMxXxp...,False,https://www.yelp.com/biz/ralphs-pizza-and-fran...,46,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 39.88254, 'longitude': -75.05772}",[],$$,"{'address1': '520 Station Ave', 'address2': ''...",+18565470030,(856) 547-0030,12500.200589
996,XF8ux9tZaXH7-gci12bejA,lees-hoagie-house-marlton-3,Lee's Hoagie House,https://s3-media2.fl.yelpcdn.com/bphoto/Rtk6gO...,False,https://www.yelp.com/biz/lees-hoagie-house-mar...,30,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.0,"{'latitude': 39.91473098098806, 'longitude': -...","[delivery, pickup]",$,"{'address1': '230 N Maple Ave', 'address2': 'S...",+18569880600,(856) 988-0600,21372.931996
997,OG8er33dEnzM1Bc9nErUmA,1-stop-pizza-philadelphia,1 Stop Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/B5DTIV...,False,https://www.yelp.com/biz/1-stop-pizza-philadel...,7,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"{'latitude': 40.00939, 'longitude': -75.15067}","[delivery, pickup]",$,"{'address1': '3701 Germantown Ave', 'address2'...",+12152251695,(215) 225-1695,6425.551184
998,AOdw-xjhc-ZRVhJzAgJxSw,vincents-pizza-rockledge,Vincent's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/R49-Vf...,False,https://www.yelp.com/biz/vincents-pizza-rockle...,88,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.08164, 'longitude': -75.0912}",[delivery],$,"{'address1': '412 Huntingdon Pike', 'address2'...",+12156639963,(215) 663-9963,15752.472654
999,yDHINEt7By47My7x4Mb8oA,the-gypsy-saloon-west-conshohocken,The Gypsy Saloon,https://s3-media1.fl.yelpcdn.com/bphoto/XrfFNw...,False,https://www.yelp.com/biz/the-gypsy-saloon-west...,221,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 40.0689154, 'longitude': -75.3125...","[delivery, pickup]",$$,"{'address1': '128 Ford St', 'address2': '', 'a...",+16108288494,(610) 828-8494,17577.668093
